# MPIA Arxiv on Deck 2: Debugging notebook

In this notebook, I keep some first order commands for diagnostic of issues with papers.
Main definitions are taken from the main notebook.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

# Some useful definitions.
class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

We get the author list from the MPIA website

In [2]:
# Getting the list of authors can take sometimes (internet connection)
# Caching the MPIA author list to avoid running this line every time we restart the kernel.
import yaml
try:
    with open('tmp_mpia_authors.yml', 'r') as fin:
        mpia_authors = yaml.load(fin, yaml.BaseLoader)
    print("`mpia.get_mpia_mitarbeiter_list()`: restored from cache")
except FileNotFoundError:
    print("`mpia.get_mpia_mitarbeiter_list()`: cannot be restored from cache.")
    # get list from MPIA website
    # it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
    mpia_authors = mpia.get_mpia_mitarbeiter_list()
    with open('tmp_mpia_authors.yml', 'w') as fout:
        fout.write(yaml.dump(mpia_authors))

`mpia.get_mpia_mitarbeiter_list()`: restored from cache


In [3]:
# Get family names only
data = [k[-1] for k in mpia_authors]

# clean -- Having titles is something new at MPIA
data = [k.replace('Dr. ', '').strip() for k in data]

filtered_data = list(filter(mpia.filter_non_scientists, data))

name_variations = filter(lambda x: x is not None,
                         [mpia.consider_variations(name) for name in filtered_data])
mitarbeiter_list = sorted(filtered_data + list(name_variations))

lst = [(mpia.get_special_corrections(mpia.get_initials(name)), name) for name in mitarbeiter_list]
lst = [(mpia.family_name_from_initials(k[0]), k[0], k[1]) for k in lst]
lst = sorted(lst, key=lambda x: x[0])
lst = set(lst)

We get the paper to debug

In [4]:
which = "2302.12805"
paper = get_paper_from_identifier(which)
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2302.12805-b31b1b.svg)](https://arxiv.org/abs/2302.12805) | **JADES: Discovery of extremely high equivalent width Lyman-alpha emission from a faint galaxy within an ionized bubble at $z=7.3$**  |
|| Aayush Saxena, et al. |
|*Appeared on*| *2023-02-24*|
|*Comments*| *12 pages, 7 figures, submitted to Astronomy & Astrophysics, comments welcome!*|
|**Abstract**| We report the discovery of a remarkable Ly$\alpha$ emitting galaxy at$z=7.278$, JADES-GS+53.16746-27.7720 (shortened to JADES-GS-z7-LA), withEW$_0$(Ly$\alpha$) $\approx400 \pm 90$A and UV magnitude $-16.7$. Thespectroscopic redshift is confirmed via rest-frame optical lines [O II],H$\beta$ and [O III] in its JWST/NIRSpec Micro-Shutter Assembly (MSA) spectrum.The Ly$\alpha$ line is detected in both lower resolution ($R\sim100$) PRISM aswell as medium resolution ($R\sim1000$) G140M grating spectra. The Ly$\alpha$FWHM in the grating is $\approx360$ km s$^{-1}$ and the line peaks within $120$km s$^{-1}$ of the systemic redshift, indicative of very little neutral gas ordust within the galaxy. We estimate the Ly$\alpha$ escape fraction to be$\sim100\%$. JADES-GS-z7-LA has a [O III]/[O II] ratio (O32) of $8.8 \pm 1.1$and ([O III]+[O II])/H$\beta$ ratio (R23) of $9.6\pm2.2$, consistent with lowmetallicity and high ionization parameters. Deep NIRCam imaging also revealed aclose companion source (separated by $0.23''$), which exhibits similarphotometry to that of JADES-GS-z7-LA, with a photometric excess in the F410MNIRCam image consistent with [O III]+H$\beta$ emission at the same redshift.The spectral energy distribution of JADES-GS-z7-LA indicates a `bursty'star-formation history, with a low stellar mass of $10^{7.15}$ M$_\odot$. Theonly explanation of the high EW Ly$\alpha$ emission seen in JADES-GS-z7-LA isif it resides in an ionized bubble with radius $\gtrsim 3$ pMpc. Owing to thefaint nature of JADES-GS-z7-LA, we show that it is incapable of single-handedlyionizing a bubble large enough. Therefore, we suggest that JADES-GS-z7-LA (andpossibly the companion source) may be a part of a larger overdensity,presenting direct evidence of overlapping ionized bubbles at $z>7$, allowing usto study the process of reionization across both small and large scales.|

In [5]:
# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in lst]

hl_authors = highlight_authors_in_list(paper['authors'], hl_list, verbose=True)
matches = [(hl, orig) for hl, orig in zip(hl_authors, paper['authors']) if 'mark' in hl]
if not matches:
    warnings.warn(AffiliationWarning("WARNING: This paper does not seem to have MPIA authors."))
paper['authors'] = hl_authors
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2302.12805-b31b1b.svg)](https://arxiv.org/abs/2302.12805) | **JADES: Discovery of extremely high equivalent width Lyman-alpha emission from a faint galaxy within an ionized bubble at $z=7.3$**  |
|| Aayush Saxena, et al. -- incl., <mark>Anna De Graaff</mark>, <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2023-02-24*|
|*Comments*| *12 pages, 7 figures, submitted to Astronomy & Astrophysics, comments welcome!*|
|**Abstract**| We report the discovery of a remarkable Ly$\alpha$ emitting galaxy at$z=7.278$, JADES-GS+53.16746-27.7720 (shortened to JADES-GS-z7-LA), withEW$_0$(Ly$\alpha$) $\approx400 \pm 90$A and UV magnitude $-16.7$. Thespectroscopic redshift is confirmed via rest-frame optical lines [O II],H$\beta$ and [O III] in its JWST/NIRSpec Micro-Shutter Assembly (MSA) spectrum.The Ly$\alpha$ line is detected in both lower resolution ($R\sim100$) PRISM aswell as medium resolution ($R\sim1000$) G140M grating spectra. The Ly$\alpha$FWHM in the grating is $\approx360$ km s$^{-1}$ and the line peaks within $120$km s$^{-1}$ of the systemic redshift, indicative of very little neutral gas ordust within the galaxy. We estimate the Ly$\alpha$ escape fraction to be$\sim100\%$. JADES-GS-z7-LA has a [O III]/[O II] ratio (O32) of $8.8 \pm 1.1$and ([O III]+[O II])/H$\beta$ ratio (R23) of $9.6\pm2.2$, consistent with lowmetallicity and high ionization parameters. Deep NIRCam imaging also revealed aclose companion source (separated by $0.23''$), which exhibits similarphotometry to that of JADES-GS-z7-LA, with a photometric excess in the F410MNIRCam image consistent with [O III]+H$\beta$ emission at the same redshift.The spectral energy distribution of JADES-GS-z7-LA indicates a `bursty'star-formation history, with a low stellar mass of $10^{7.15}$ M$_\odot$. Theonly explanation of the high EW Ly$\alpha$ emission seen in JADES-GS-z7-LA isif it resides in an ionized bubble with radius $\gtrsim 3$ pMpc. Owing to thefaint nature of JADES-GS-z7-LA, we show that it is incapable of single-handedlyionizing a bubble large enough. Therefore, we suggest that JADES-GS-z7-LA (andpossibly the companion source) may be a part of a larger overdensity,presenting direct evidence of overlapping ionized bubbles at $z>7$, allowing usto study the process of reionization across both small and large scales.|

We get the (TeX) source
* retrieve the tarball
* find the main tex file and parse it
* parse for affiliations (but debugging so we do not stop if not found)
* generate the the output markdown

In [6]:
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'

if not os.path.isdir(folder):
    folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')

try:
    doc = latex.LatexDocument(folder, validation=validation)    
except AffiliationError as affilerror:
    msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
    print(msg)

# Hack because sometimes author parsing does not work well
if (len(doc.authors) != len(paper['authors'])):
    doc._authors = paper['authors']
if (doc.abstract) in (None, ''):
    doc._abstract = paper['abstract']

doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
doc.highlight_authors_in_list(hl_list, verbose=True)

full_md = doc.generate_markdown_text()

In [8]:
try:
    bibdata = latex_bib.LatexBib.from_doc(doc)
    full_md = latex_bib.replace_citations(full_md, bibdata)
except Exception as e:
    print(e)

<re.Match object; span=(5095, 5108), match='\\citep{bun23}'>
<re.Match object; span=(5346, 5366), match='\\citep{fur16, has19}'>
<re.Match object; span=(5389, 5409), match='\\citep{oes15, sta17}'>
<re.Match object; span=(5432, 5459), match='\\citep{pen16, lap17, sta17}'>
<re.Match object; span=(5484, 5511), match='\\citep{van11, mai15, car17}'>
<re.Match object; span=(5554, 5567), match='\\citep{val22}'>
<re.Match object; span=(5222, 5235), match='\\citet{tan23}'>
<re.Match object; span=(5281, 5295), match='\\citet{end22c}'>


In [9]:
print(doc.source)

\documentclass[longauth]{aa}
\usepackage{graphicx}
\usepackage{txfonts}
\usepackage{hyperref}
\hypersetup{
    colorlinks=true,
    linkcolor=blue,
    citecolor=blue,
    filecolor=magenta,
    urlcolor=cyan,
    }
\usepackage{textgreek}
\newcommand{\vdag}{(v)^\dagger}
\newcommand\aastex{AAS\TeX}
\newcommand\latex{La\TeX}
\newcommand{\lya}{Ly\textalpha}
\newcommand{\ha}{H\textalpha}
\newcommand{\hb}{H\textbeta}
\newcommand{\hg}{H\textgamma}
\newcommand{\oiii}{[O \textsc{iii}]}
\newcommand{\oii}{[O \textsc{ii}]}
\newcommand{\hii}{H \textsc{ii}}
\newcommand{\xiion}{$\xi_\mathrm{ion}$}
\newcommand{\fesc}{$f_\mathrm{esc}$}
\newcommand{\flux}{erg s$^{-1}$ cm$^{-2}$}
\newcommand{\sfr}{$M_\odot$ yr$^{-1}$}
\newcommand{\kms}{km s$^{-1}$}
\begin{document}
\title{JADES: Discovery of extremely high equivalent width Lyman-alpha emission from a faint galaxy within an ionized bubble at $z=7.3$}
\titlerunning{Discovery of an extreme LAE at $z=7.3$}
\author{
Aayush Saxena\inst{1,2}\thanks{E-mail: aay

In [10]:
from IPython.display import Markdown
Markdown(full_md)

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\lya}{Ly\textalpha}$
$\newcommand{\ha}{H\textalpha}$
$\newcommand{\hb}{H\textbeta}$
$\newcommand{\hg}{H\textgamma}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\xiion}{\xi_\mathrm{ion}}$
$\newcommand{\fesc}{f_\mathrm{esc}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}}$
$\newcommand{\sfr}{M_\odot yr^{-1}}$
$\newcommand{\kms}{km s^{-1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\lya}{Ly\textalpha}$
$\newcommand{\ha}{H\textalpha}$
$\newcommand{\hb}{H\textbeta}$
$\newcommand{\hg}{H\textgamma}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\xiion}{\xi_\mathrm{ion}}$
$\newcommand{\fesc}{f_\mathrm{esc}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}}$
$\newcommand{\sfr}{M_\odot yr^{-1}}$
$\newcommand{\kms}{km s^{-1}}$</div>



<div id="title">

# JADES: Discovery of extremely high equivalent width Lyman-alpha emission from a faint galaxy within an ionized bubble at $z=7.3$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.12805-b31b1b.svg)](https://arxiv.org/abs/2302.12805) _12 pages, 7 figures, submitted to Astronomy & Astrophysics, comments welcome!_

</div>
<div id="authors">

Aayush Saxena, et al. -- incl., <mark>Anna De Graaff</mark>, <mark>Hans-Walter Rix</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of a remarkable Ly $\alpha$ emitting galaxy at $z=7.278$ , JADES-GS+53.16746-27.7720 (shortened to JADES-GS-z7-LA), with EW $_0$ (Ly $\alpha$ ) $\approx400 \pm 90$ Å and UV magnitude $-16.7$ . The spectroscopic redshift is confirmed via rest-frame optical lines [ O ii ] , H $\beta$ and [ O iii ] in its JWST/NIRSpec Micro-Shutter Assembly (MSA) spectrum. The Ly $\alpha$ line is detected in both lower resolution ( $R\sim100$ ) PRISM as well as medium resolution ( $R\sim1000$ ) G140M grating spectra. The Ly $\alpha$ FWHM in the grating is $\approx360$ km s $^{-1}$ and the line peaks with [ O ii ] in $120$ km s $^{-1}$ of the systemic redshift, indicative of very little neutral gas or dust within the galaxy. We estimate the Ly $\alpha$ escape fraction to be $\sim100\%$ . JADES-GS-z7-LA has a [ O iii ] / [ O ii ] ratio (O32) of $8.8 \pm 1.1$ and ( [ O iii ] + [ O ii ] )/H $\beta$ ratio (R23) of $9.6\pm2.2$ , consistent with low metallicity and high ionization parameters. Deep NIRCam imaging also revealed a close companion source (separated by $0.23"$ ), which exhibits similar photometry to that of JADES-GS-z7-LA, with a photometric excess in the F410M NIRCam image consistent with [ O iii ] +H $\beta$ emission at the same redshift. The spectral energy distribution of JADES-GS-z7-LA indicates a `bursty' star-formation history, with a low stellar mass of $10^{7.15}$ M $_\odot$ . The only explanation of the high EW Ly $\alpha$ emission seen in JADES-GS-z7-LA is if it resides in an ionized bubble with radius $\gtrsim 3$ pMpc. Owing to the faint nature of JADES-GS-z7-LA, we show that it is incapable of single-handedly ionizing a bubble large enough. Therefore, we suggest that JADES-GS-z7-LA (and possibly the companion source) may be a part of a larger overdensity, presenting direct evidence of overlapping ionized bubbles at $z>7$ , allowing us to study the process of reionization across both small and large scales.

</div>

<div id="div_fig1">

<img src="tmp_2302.12805/./figs/GS_LA7_Lya_R1000_vel.png" alt="Fig1" width="100%"/>

**Figure 1. -** Velocity shift of the $\lya$ emission line in the G140M NIRSpec spectrum with a resolution of $R\sim1000$ compared to the systemic redshift ($\Delta v=0$\kms) traced by the $\oiii$ and $\hb$ lines. The emission line appears to be symmetric, centred very close to the systemic redshift with an offset of $120\pm80$\kms, with flux bluewards of the peak. The observed line profile is indicative of significant $\lya$ photon escape thanks to virtually no neutral gas along the line-of-sight in the galaxy. (*fig:R1000_spec*)

</div>
<div id="div_fig2">

<img src="tmp_2302.12805/./figs/GS_LA7_vel_comparison_v2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Distribution of $\lya$ EW and its velocity offset from the systemic redshift for JADES-GS-z7-LA (star), compared with GN-z11  ([Bunker, et. al 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230207256B)) (diamond), $z>7$ LAEs identified from CEERS by  ([Tang, et. al (2023)](https://ui.adsabs.harvard.edu/abs/2023arXiv230107072T)) (circles) and other known $z>7$ LAEs from the  ([Endsley, et. al (2022)](https://ui.adsabs.harvard.edu/abs/2022MNRAS.517.5642E))  compilation (squares), which include sources from the ALMA REBELS survey amongst others such as B14-65666 at $z=7.15$ ([Furusawa, et. al 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...822...46F), [Hashimoto, et. al 2019](https://ui.adsabs.harvard.edu/abs/2019PASJ...71...71H)) , EGS-zs8-1 at $z=7.72$ ([Oesch, et. al 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...804L..30O), [Stark, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.464..469S)) , COS-zs7-1 at $z=7.15$ ([Pentericci, et. al 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...829L..11P), [Laporte, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...851...40L), [Stark, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.464..469S))  and BDF-3299 at $z=7.11$ ([Vanzella, et. al 2011](https://ui.adsabs.harvard.edu/abs/2011ApJ...730L..35V), [Maiolino, et. al 2015](https://ui.adsabs.harvard.edu/abs/2015MNRAS.452...54M), [Carniani, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017A&A...605A..42C)) . We also include the $z=7.68$ galaxy Y002  ([Valentino, et. al 2022](https://ui.adsabs.harvard.edu/abs/2022ApJ...929L...9V)) . JADES-GS-z7-LA lines up well with the observed strong anti-correlation between the strength of the $\lya$ emission line and its velocity offset compared to systemic redshift, and its faint UV magnitude and high $\lya$ EW compared to other $z>7$ LAEs stand out. (*fig:vel_comp*)

</div>
<div id="div_fig3">

<img src="tmp_2302.12805/./figs/LAE-companion-Bagpipes-fits-v0.7.png" alt="Fig3" width="100%"/>

**Figure 3. -** Best-fitting SEDs of JADES-GS-z7-LA and its companion source, fixing the redshift to be at $z=7.278$ for both objects. We find that a good fit is obtained for the companion source at the same redshift as JADES-GS-z7-LA, which in particular can explain the photometric excess observed in the F410M band (dashed oval). Overall, JADES-GS-z7-LA has a `bluer' SED, which is reflected in the preference for a bursty star-formation history when compared to the companion source. (*fig:SED*)

</div>